In [1]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
query = '''SELECT f.rental_duration, f.rental_rate, f.length, f.rating, fc.category_id FROM film f
JOIN film_category fc
USING (film_id);'''

data = pd.read_sql_query(query, engine)
data.head()

,rental_duration,rental_rate,length,rating,category_id
0,6,0.99,113,PG,1
1,3,4.99,129,R,1
2,5,2.99,168,NC-17,1
3,6,0.99,68,NC-17,1
4,6,2.99,129,G,1


In [4]:
# Create temp table

sql = '''CREATE TEMPORARY TABLE rentedmay AS (
  SELECT film_id, f.title FROM rental r
JOIN inventory i USING (inventory_id)
JOIN film f USING (film_id)
WHERE rental_date LIKE '%%2005-05%%'
GROUP BY film_id
ORDER BY rental_date
);'''
engine.execute(sql)

In [5]:
# Use temp table with left join to get NaNs in Column title for movies that havn't been rented in May

query2 = '''SELECT f.rental_duration, f.rental_rate, f.length, f.rating, fc.category_id, film_id, re.title FROM film f
JOIN film_category fc
USING (film_id)
LEFT JOIN rentedmay re
USING (film_id)
ORDER BY film_id;'''

data2 = pd.read_sql_query(query2, engine)

data2.head()

,rental_duration,rental_rate,length,rating,category_id,film_id,title
0,6,0.99,86,PG,6,1,ACADEMY DINOSAUR
1,3,4.99,48,G,11,2,None
2,7,2.99,50,NC-17,6,3,ADAPTATION HOLES
3,5,2.99,117,G,11,4,AFFAIR PREJUDICE
4,6,2.99,130,G,8,5,AFRICAN EGG


In [6]:
# data2['title'] = data2['title'].fillna('FALSE')

In [12]:
# data2['title'] = data2['title'].replace(['None'], 'False')
# data2
import numpy as np

data.isna().sum()
data2.info(verbose=True)
data2['title'] = data2['title'].replace(np.nan, False)
data2

SyntaxError: 'return' outside function (2138429453.py, line 12)

In [9]:
import numpy as np
# data2['May'] = np.where((data2['title'] != 'False'), 'TRUE', 'FALSE')
# data2

data2['May']=np.where((data2['title'].isin(['False'])),'True', 'False')
data2

,rental_duration,rental_rate,length,rating,category_id,film_id,title,May
0,6,0.99,86,PG,6,1,ACADEMY DINOSAUR,False
1,3,4.99,48,G,11,2,False,False
2,7,2.99,50,NC-17,6,3,ADAPTATION HOLES,False
3,5,2.99,117,G,11,4,AFFAIR PREJUDICE,False
4,6,2.99,130,G,8,5,AFRICAN EGG,False
...,...,...,...,...,...,...,...,...
995,6,0.99,183,G,6,996,False,False
996,4,0.99,179,NC-17,12,997,False,False
997,6,0.99,105,NC-17,11,998,ZHIVAGO CORE,False
998,5,2.99,101,R,3,999,ZOOLANDER FICTION,False


In [ ]:
data2['title'].value_counts()